Merge IN and OUT prediction together
--

In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from xgboost import plot_importance
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split
import math, sys
from matplotlib import pyplot as plt
%matplotlib inline

### Hourly

In [2]:
df_out = pd.read_csv('./000Deliverables/20181106_XGB_9_70_3_03_OUT/Clean_Daily2Hourly_V311_APE_XGB_9_70_3_03_OUT.csv')

df_out['TRFFC_DTE'] = pd.to_datetime(df_out['Date'], infer_datetime_format=True)
df_out['Date'] = df_out['TRFFC_DTE'].dt.strftime('%Y-%m-%d')

df_out = df_out[['STR_ID', 'Date', 'SALEHOUR','IN_Clean', 'pred_hourly']]

df_out.rename(columns={'pred_hourly':'out_pred', 'IN_Clean':'OUT_true'}, inplace=True)

In [3]:
df_in = pd.read_csv('./000Deliverables/20181106_XGB_9_70_3_03_IN/Clean_Daily2Hourly_V311_APE_XGB_9_70_3_03_IN.csv')

df_in['TRFFC_DTE'] = pd.to_datetime(df_in['Date'], infer_datetime_format=True)
df_in['Date'] = df_in['TRFFC_DTE'].dt.strftime('%Y-%m-%d')

df_in = df_in[['STR_ID', 'Date', 'SALEHOUR','IN_Clean', 'pred_hourly']]

df_in.rename(columns={'pred_hourly':'in_pred', 'IN_Clean':'IN_true'}, inplace=True)

In [4]:
df_in.head(3)

,STR_ID,Date,SALEHOUR,IN_true,in_pred
0,7,2018-06-03,0,0.0,0.0
1,7,2018-06-03,1,0.0,0.0
2,7,2018-06-03,2,0.0,0.0


In [5]:
df_out.head(3)

,STR_ID,Date,SALEHOUR,OUT_true,out_pred
0,7,2018-06-03,0,0.0,0.0
1,7,2018-06-03,1,0.0,0.0
2,7,2018-06-03,2,0.0,0.0


In [6]:
df_all = df_in.merge(df_out, on=['STR_ID', 'Date', 'SALEHOUR'])

In [7]:
df_all['TRFFC_DTE'] = pd.to_datetime(df_all.Date + ' ' + df_all['SALEHOUR'].astype('str') + ':00:00',
                                    infer_datetime_format=True)

In [8]:
df_all = df_all[['STR_ID', 'TRFFC_DTE', 'IN_true', 'in_pred', 'OUT_true', 'out_pred']]

In [9]:
df_all = df_all.assign(model_date=['2018-11-06' for i in range(len(df_all))])

In [10]:
df_all.to_csv('./000Deliverables/20181106_XGB_9_70_3_03_hourly_All.csv', index=False)

In [11]:
df_all.head(3)

,STR_ID,TRFFC_DTE,IN_true,in_pred,OUT_true,out_pred,model_date
0,7,2018-06-03 00:00:00,0.0,0.0,0.0,0.0,2018-11-06
1,7,2018-06-03 01:00:00,0.0,0.0,0.0,0.0,2018-11-06
2,7,2018-06-03 02:00:00,0.0,0.0,0.0,0.0,2018-11-06


### Daily

In [12]:
df_out = pd.read_csv('./000Deliverables/20181106_XGB_9_70_3_03_OUT/Clean_Daily_V311_APE_XGB_9_70_3_03_OUT.csv')

df_out['TRFFC_DTE'] = pd.to_datetime(df_out['Date'], infer_datetime_format=True)
df_out['Date'] = df_out['TRFFC_DTE'].dt.strftime('%Y-%m-%d')

df_out = df_out[['STR_ID', 'Date', 'VSTR_IN_CNT', 'VSTR_IN_CNT_pred']]

df_out.rename(columns={'VSTR_IN_CNT_pred':'out_pred', 'VSTR_IN_CNT':'OUT_true'}, inplace=True)

In [13]:
df_in = pd.read_csv('./000Deliverables/20181106_XGB_9_70_3_03_IN/Clean_Daily_V311_APE_XGB_9_70_3_03_IN.csv')

df_in['TRFFC_DTE'] = pd.to_datetime(df_in['Date'], infer_datetime_format=True)
df_in['Date'] = df_in['TRFFC_DTE'].dt.strftime('%Y-%m-%d')

df_in = df_in[['STR_ID', 'Date', 'VSTR_IN_CNT', 'VSTR_IN_CNT_pred']]

df_in.rename(columns={'VSTR_IN_CNT_pred':'in_pred', 'VSTR_IN_CNT':'IN_true'}, inplace=True)

In [14]:
df_in.head(3)

,STR_ID,Date,IN_true,in_pred
0,7,2018-06-03,3881.0,3261.275391
1,7,2018-06-04,2426.0,2267.406738
2,7,2018-06-05,2390.0,2147.245117


In [15]:
df_out.head(3)

,STR_ID,Date,OUT_true,out_pred
0,7,2018-06-03,3508.0,3149.876709
1,7,2018-06-04,2236.0,2134.440674
2,7,2018-06-05,2201.0,2040.486206


In [16]:
df_all = df_in.merge(df_out, on=['STR_ID', 'Date'])

In [17]:
df_all = df_all[['STR_ID', 'Date', 'IN_true', 'in_pred', 'OUT_true', 'out_pred']]

In [18]:
df_all = df_all.assign(model_date=['2018-11-06' for i in range(len(df_all))])

In [19]:
df_all.to_csv('./000Deliverables/20181106_XGB_9_70_3_03_daily_All.csv', index=False)

In [20]:
df_all.head(3)

,STR_ID,Date,IN_true,in_pred,OUT_true,out_pred,model_date
0,7,2018-06-03,3881.0,3261.275391,3508.0,3149.876709,2018-11-06
1,7,2018-06-04,2426.0,2267.406738,2236.0,2134.440674,2018-11-06
2,7,2018-06-05,2390.0,2147.245117,2201.0,2040.486206,2018-11-06
